In [1]:
from google.colab import files
uploaded=files.upload()

Saving churn.csv to churn (2).csv


In [2]:
!pip install pyspark
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import numpy as np

from pyspark.ml.classification import GBTClassifier
import findspark
findspark.init()
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.sql import SparkSession
from pyspark.sql import Row
import collections
from pyspark import SparkConf,SparkContext

In [4]:
df = pd.read_csv("churn.csv")
del df["Unnamed: 0"]
df

,Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Churn
0,Cameron Williams,42.0,11066.80,0,7.22,8.0,1
1,Kevin Mueller,41.0,11916.22,0,6.50,11.0,1
2,Eric Lozano,38.0,12884.75,0,6.67,12.0,1
3,Phillip White,42.0,8010.76,0,6.71,10.0,1
4,Cynthia Norton,37.0,9191.58,0,5.56,9.0,1
...,...,...,...,...,...,...,...
895,Paul Miller,42.0,12800.82,1,3.62,8.0,0
896,Natalie Hodges,52.0,9893.92,0,6.91,7.0,0
897,Ana Smith,45.0,12056.18,0,5.46,4.0,0
898,Justin Leonard,51.0,6517.93,1,5.47,10.0,0


In [5]:
df.isnull().sum()

Names              0
Age                0
Total_Purchase     0
Account_Manager    0
Years              0
Num_Sites          0
Churn              0
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Names            900 non-null    object 
 1   Age              900 non-null    float64
 2   Total_Purchase   900 non-null    float64
 3   Account_Manager  900 non-null    int64  
 4   Years            900 non-null    float64
 5   Num_Sites        900 non-null    float64
 6   Churn            900 non-null    int64  
dtypes: float64(4), int64(2), object(1)
memory usage: 49.3+ KB


In [7]:
df.corr()

,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Churn
Age,1.000000,-0.037208,-0.014749,0.005625,-0.006070,0.085926
Total_Purchase,-0.037208,1.000000,0.015856,-0.005623,-0.003390,0.024031
Account_Manager,-0.014749,0.015856,1.000000,0.022930,0.033401,0.070611
Years,0.005625,-0.005623,0.022930,1.000000,0.051642,0.214329
Num_Sites,-0.006070,-0.003390,0.033401,0.051642,1.000000,0.525398
Churn,0.085926,0.024031,0.070611,0.214329,0.525398,1.000000


In [8]:
df.columns

Index(['Names', 'Age', 'Total_Purchase', 'Account_Manager', 'Years',
       'Num_Sites', 'Churn'],
      dtype='object')

In [9]:
for col in df[['Age', 'Total_Purchase', 'Account_Manager', 'Years','Num_Sites', 'Churn']] : 
    df[col]=df[col].astype(float)

In [10]:
df.to_csv('churn2.csv')

## Pyspark

In [11]:
from __future__ import print_function

In [12]:
spark=SparkSession.builder.getOrCreate()

In [13]:
inputlines=df = spark.read.options(header = True, inferSchema = True).csv("churn2.csv")
inputlines

DataFrame[_c0: int, Names: string, Age: double, Total_Purchase: double, Account_Manager: double, Years: double, Num_Sites: double, Churn: double]

In [14]:
df1 = inputlines.rdd.map(lambda x: x.split(","))
df1

PythonRDD[15] at RDD at PythonRDD.scala:53

In [15]:
colnames=['Age','Total_Purchase','Account_Manager','Years','Num_Sites','Churn']

In [16]:
df2 = inputlines
df2

DataFrame[_c0: int, Names: string, Age: double, Total_Purchase: double, Account_Manager: double, Years: double, Num_Sites: double, Churn: double]

In [17]:
from  pyspark.ml.feature import VectorAssembler

In [18]:
vecAssembler = VectorAssembler(inputCols=['Age','Total_Purchase','Account_Manager','Years','Num_Sites'], outputCol="features")
df2= vecAssembler.transform(df2)
df2

DataFrame[_c0: int, Names: string, Age: double, Total_Purchase: double, Account_Manager: double, Years: double, Num_Sites: double, Churn: double, features: vector]

In [19]:
df2 = df2.drop("index", 'Age','Total_Purchase','Account_Manager','Years','Num_Sites')
df2

DataFrame[_c0: int, Names: string, Churn: double, features: vector]

In [20]:
df2 = df2.withColumnRenamed("Churn", "label")
df2

DataFrame[_c0: int, Names: string, label: double, features: vector]

In [21]:
trainTest=df2.randomSplit([0.5,0.5])
trainingDF=trainTest[0]
testDF=trainTest[1]

In [22]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

In [23]:
gbt=GBTClassifier(featuresCol='features',maxIter=10)
gbt = gbt.fit(trainingDF)

In [24]:
predictions = gbt.transform(testDF)
predictions.show()

+---+------------------+-----+--------------------+--------------------+--------------------+----------+
|_c0|             Names|label|            features|       rawPrediction|         probability|prediction|
+---+------------------+-----+--------------------+--------------------+--------------------+----------+
|  2|       Eric Lozano|  1.0|[38.0,12884.75,0....|[-1.2413224089277...|[0.07708383413859...|       1.0|
|  3|     Phillip White|  1.0|[42.0,8010.76,0.0...|[1.35664746104730...|[0.93780660364570...|       0.0|
|  4|    Cynthia Norton|  1.0|[37.0,9191.58,0.0...|[1.15429583726163...|[0.90958610249070...|       0.0|
|  7|     Zachary Walsh|  1.0|[32.0,9885.12,1.0...|[0.98981721412248...|[0.87864218655291...|       0.0|
|  8|       Ashlee Carr|  1.0|[43.0,14062.6,1.0...|[-0.6389157883877...|[0.21791956324842...|       1.0|
| 11|    Bruce Phillips|  1.0|[45.0,8771.02,1.0...|[0.50264973770516...|[0.73209924201892...|       0.0|
| 13|      Nicole Olson|  1.0|[40.0,8283.32,1.0...|[-1.

In [25]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)


Accuracy: 0.8721973094170403
